# Parse Team Data

This notebook parses the scraped box scores (html files) and extracts the team statistics, which are summarizes as an entry in a pandas dataframe. This dataframe serves as the raw dataset for all team statistics from our scraped data.

The df is exported as a csv file

In [82]:
import os
import pandas as pd
from io import StringIO
import chardet
from bs4 import BeautifulSoup

SCORE_DIR = "../../data/raw/scores"
# SCORE_DIR = "../../data/raw/scraped_html/scores"

In [83]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [84]:
len(box_scores)

11022

In [85]:
def parse_html(box_score):
    with open(box_score, "rb") as f:
        try:
            r_data = f.read()
            encoding = chardet.detect(r_data)["encoding"]
            html = r_data.decode(encoding)
        except Exception as e:
            print(f"html with this title: {box_score.title()} is probably empty....", e)
            return 0
            
    soup = BeautifulSoup(html, "lxml")
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [86]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [87]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [88]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [89]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)
    #indicates theres an html that has nothing, probably need to add something to delete that code...
    if soup == 0:
        continue
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols] 
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 11022
200 / 11022
300 / 11022
400 / 11022
500 / 11022
600 / 11022
700 / 11022
800 / 11022
900 / 11022
1000 / 11022
1100 / 11022
html with this title: Data2/Scores\201604010Mem.Html is probably empty.... decode() argument 'encoding' must be str, not None
html with this title: Data2/Scores\201604010Nyk.Html is probably empty.... decode() argument 'encoding' must be str, not None
1200 / 11022
1300 / 11022
1400 / 11022
1500 / 11022
1600 / 11022
1700 / 11022
1800 / 11022
1900 / 11022
2000 / 11022
2100 / 11022
2200 / 11022
2300 / 11022
2400 / 11022
2500 / 11022
2600 / 11022
2700 / 11022
2800 / 11022
2900 / 11022
3000 / 11022
3100 / 11022
3200 / 11022
3300 / 11022
3400 / 11022
3500 / 11022
3600 / 11022
3700 / 11022
3800 / 11022
3900 / 11022
4000 / 11022
4100 / 11022
4200 / 11022
4300 / 11022
4400 / 11022
4500 / 11022
4600 / 11022
4700 / 11022
4800 / 11022
4900 / 11022
5000 / 11022
5100 / 11022
5200 / 11022
5300 / 11022
5400 / 11022
5500 / 11022
5600 / 11022
5700 / 11022
5800 / 11022
590

In [90]:
games_df = pd.concat(games, ignore_index=True)

In [91]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22035,240.0,240.0,44.0,83.0,0.530,13.0,26.0,0.500,12.0,19.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22036,240.0,240.0,46.0,89.0,0.517,11.0,30.0,0.367,25.0,27.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22037,240.0,240.0,31.0,83.0,0.373,8.0,29.0,0.276,21.0,30.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False
22038,240.0,240.0,49.0,92.0,0.533,20.0,42.0,0.476,22.0,28.0,...,66.7,30.8,175.0,146.0,UTA,137,1,2024,2024-02-15,True


In [92]:
games_df.to_csv("../../data/raw/nba_games_runtime.csv")